In [1]:
import tensorflow as tf
import numpy as np
from utils import *

 I was having some confusion so I tried various slicing operations and reshaping

In [2]:
a=np.array([[1,2],[3,4]])
print(a[:,0])
print(a[:,0].shape)
print(a[:,0].reshape(-1,1))
a1=np.zeros((2,2))
a1[:,0]=a[:,0]
print(a1)

[1 3]
(2,)
[[1]
 [3]]
[[1. 0.]
 [3. 0.]]


In [3]:
data=open('./dinos.txt').read()
data=data.lower()
chars=sorted(list(set(data)))
vocab_size=len(chars)
print('There are %d unique characters in your data.' % (vocab_size))
print(chars)

There are 27 unique characters in your data.
['\n', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [4]:
examples=[e.strip() for e in data.split('\n')]
examples

['aachenosaurus',
 'aardonyx',
 'abdallahsaurus',
 'abelisaurus',
 'abrictosaurus',
 'abrosaurus',
 'abydosaurus',
 'acanthopholis',
 'achelousaurus',
 'acheroraptor',
 'achillesaurus',
 'achillobator',
 'acristavus',
 'acrocanthosaurus',
 'acrotholus',
 'actiosaurus',
 'adamantisaurus',
 'adasaurus',
 'adelolophus',
 'adeopapposaurus',
 'aegyptosaurus',
 'aeolosaurus',
 'aepisaurus',
 'aepyornithomimus',
 'aerosteon',
 'aetonyxafromimus',
 'afrovenator',
 'agathaumas',
 'aggiosaurus',
 'agilisaurus',
 'agnosphitys',
 'agrosaurus',
 'agujaceratops',
 'agustinia',
 'ahshislepelta',
 'airakoraptor',
 'ajancingenia',
 'ajkaceratops',
 'alamosaurus',
 'alaskacephale',
 'albalophosaurus',
 'albertaceratops',
 'albertadromeus',
 'albertavenator',
 'albertonykus',
 'albertosaurus',
 'albinykus',
 'albisaurus',
 'alcovasaurus',
 'alectrosaurus',
 'aletopelta',
 'algoasaurus',
 'alioramus',
 'aliwalia',
 'allosaurus',
 'almas',
 'alnashetri',
 'alocodon',
 'altirhinus',
 'altispinax',
 'alvarez

In [5]:
char_to_ix={c:i for i,c in enumerate(chars)}
ix_to_char={i:c for i,c in enumerate(chars)}
print(char_to_ix)
print(ix_to_char)

{'\n': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26}
{0: '\n', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z'}


In [6]:
def clip(gradients,maxval):
    
    dWax=gradients['dWax']
    dWaa=gradients['dWaa']
    dWya=gradients['dWya']
    dba=gradients['dba']
    dby=gradients['dby']
    
    for gradient in [dWax,dWaa,dWya,dba,dby]:
        np.clip(gradient,-maxval,maxval,out=gradient)
        
    gradients['dWax']=dWax
    gradients['dWaa']=dWaa
    gradients['dWya']=dWya
    gradients['dba']=dba
    gradients['dby']=dby
    
    return gradients

In [7]:
def sample(parameters,generated_size=50):
    
    Wax=parameters['Wax']
    Waa=parameters['Waa']
    Wya=parameters['Wya']
    ba=parameters['ba']
    by=parameters['by']
    
    n_a,n_x=Wax.shape
    
    x=np.zeros((n_x,1))
    a0=np.zeros((n_a,1))
    a_prev=a0
    
    idx=-1
    
    indices=[] # sampled indices
    while(len(indices)<=generated_size and idx!=char_to_ix['\n']):
        a_next=np.tanh(np.dot(Wax,x)+np.dot(Waa,a_prev)+ba) # a_next=(n_a,1)
        y=softmax(np.dot(Wya,a_next)+by) # y->(n_y,1)
        
        a_prev=a_next # setting the next hidden state
        
        idx=np.random.choice(np.arange(len(y)),p=y.ravel()) # Randomly choosing an index from the softmax output
        indices.append(idx)
        
        x=np.zeros((n_x,1))
        x[idx]=1  # Creating a one-hot vector by setting the calculated index to 1 and rest 0
    
    if(len(indices)>generated_size):
        indices.append(char_to_ix['\n']) # if by chance idx is never choosen to be the index of '\n' then after the allowed 
                                        # generated_size we append that index to indices and return indices
        
    return indices

In [8]:
def optimize(X,Y,parameters,a_prev,learning_rate=0.01):
    
    loss,cache=rnn_forward(parameters,X,Y,a_prev)
    
    gradients,a=rnn_backward(X,Y,parameters,cache)
    
    gradients=clip(gradients,5)
    
    parameters=update_parameters(parameters,gradients,lr=learning_rate)
    
    return loss,gradients,a[:,-1].reshape(-1,1)

In [9]:
def model(data, char_to_ix, ix_to_char, num_iterations, n_a=64, dino_names=7, vocab_size=27):
    n_x,n_y=vocab_size,vocab_size
    
    parameters=initialize_parameters(n_a,n_x,n_y)
    
    examples=[e.strip() for e in data]
    
    a_prev=np.zeros((n_a,1))
    
    for i in range(num_iterations):
        idx=i%len(examples)
        
        single_example=examples[idx]
        single_example_chars=[c for c in single_example]
        single_example_ix=[char_to_ix[c] for c in single_example]
        X=[None]+single_example_ix
        
        Y=single_example_ix + [char_to_ix['\n']]
        
        try:
            assert a_prev.shape == (n_a, 1)
            loss, gradients, a_prev = optimize(X, Y, parameters, a_prev)
#             assert a_prev.shape == (n_a, 1)
            
            if i % 2000 == 0:
                print("Current iteration: %d Current loss: %f" % (i, loss))
                
                for name in range(dino_names):
                    sampled_indices = sample(parameters)
                    dino_name = get_sample(sampled_indices, ix_to_char)
                    print(dino_name.replace('\n', ''))
                print('\n')
        
        except AssertionError:
            print(f"Assertion error occurred at iteration {i}, with example: {single_example}, input {X}, label {Y} and a_prev's shape {a_prev.shape}")
            break
    
    return parameters

In [10]:
parameters=model(data.split('\n'),char_to_ix,ix_to_char,22000)

Current iteration: 0 Current loss: 46.142329
Eowqfcpohikg
Tdaciq

Rv
Yuvuwzfpwvdqhzektdc
Efffpgppvmyxmcoayxvltccarj
Eltsuojtlmtczzvxsosxbndsmaocoproddbfrlmeywb


Current iteration: 2000 Current loss: 26.611107
Erhonneisaideyos
Yelisaurus
Kvhaoptodhodotaurus
Erderinloo
Odrucotomkuaurus
Gos
S


Current iteration: 4000 Current loss: 24.699738
Vos
Keicbmurosenmeruatnllenengoatronognoeqiwtecelosauru
Onfngmaaptenotosaurus
Norin
Edoentorinateomotatoenonoma
S
Acosthoosaurus


Current iteration: 6000 Current loss: 26.759157
Ontoributusaurus
Ennic
Toraurun
Urus
Toraurus
Dptisaurus
Urus


Current iteration: 8000 Current loss: 12.884867
Ceros
Tol
Ogosaurus
Koshxulus
Culaurus
Urakludeuconos
Ichulscumuchupecrontocheyhus


Current iteration: 10000 Current loss: 25.319214
Onangis
Gons
Onikonosaurus
Opaurus
Lona
Yius
Tentonaldonp


Current iteration: 12000 Current loss: 48.626472
Mis
Ia
Liallorhbun
Mis
Ilanaurusallgetabmholus
Plinianlothrgtumhoosaoda
Eranaurtiurosibole


Current iteration: 14000 Curren